# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

So the easiest way to get your data is to use some unix tools such as `wget` to fetch it from a url, and then `gunzip` to extract it if it's zipped. 

Parallel corpuses come in many formats. The ideal corpus comes with 2 files: `file.source` and `file.target` where `"source"` is your source language, such as `en` and `"target"` is your target language, such as `xh` (Xhosa)

Sometimes they come in a **.tmx** file a.k.a a **translation memory file**. This is an xml structure which will include the sentences in your target language and your source language in a single file. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. #

In [1]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ts"
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["ts"] = target_language

In [2]:
rm en*

In [3]:
# Downloading and unzipping our xhosa corpus
# TODO: You'll need to download & extract your own corpus here! 
# ! wget "https://github.com/vukosim/masakhane-mt-en-ts/raw/master/data/en_ts.tmx.zip" -O en-ts.tmx.zip
# ! unzip en-ts.tmx.zip
# ! ls -lh

# This is useful if you end up having to use a tmx file,
! pip install tmx2dataframe

In [3]:
# !7z x en-ts.tmx.zip

In [30]:
!sed '154934,154937d' ../data/en_ts.tmx9 > ../data/en_ts.tmx0

In [29]:
!sed -n '154934,154937p;154937q' ../data/en_ts.tmx9

		<tu>
			<tuv xml:lang="en"><seg>The New Statesman & Society magazine puts cold water on some of the flair for rhetoric : “ The dazzled reader may well have overlooked the sketchiness of the actual hard physical evidence — or , indeed , the total absence of a basis for the scientifically egregious [ flagrant ] claim that promiscuity ‘ is encoded in the male genes and imprinted on the male brain’s circuitboard . ’ ”</seg></tuv>
			<tuv xml:lang="ts"><seg>Magazini lowu nge New Statesman & Society wu timela ku hundzeletiwa kun’wana ka ndlela leyi khavisiweke leyi vutshuburi lebyi byi hlamuseriwaka ha yona wu ku : “ Muhlayi la nga riki na vutivi a nga ha va a honise leswi tumbeleke eka vumbhoni lebyo dzika lebyi kumiweke — kumbe entiyisweni , leswaku hilaha ku heleleke ku hava xisekelo xa swilo leswi swi nga manyala hi tlhelo ra sayense swa leswaku vunghwavava ‘ byi le ka swiaki swa xitekela swa munhu wa xinuna , naswona byi tshame entsindza wa byongo bya munhu wa xinuna . ’ ”</seg></tuv>

In [28]:
## import pandas as pd
from tmx2dataframe import tmx2dataframe

# TODO:
# If your source is a translation memory file (tmx file), then the one file contains both your target and source language. If so, set tmx_file = "your file here"
# Comment if you have 2 files, instead of the tmx file
tmx_file = "../data/en_ts.tmx9"
source_file = None
target_file = None 

# Uncomment if you have 2 files and set source_file and target_file to the path of your parallel corpus files
# tmx_file = None
# source_file = 'file.src'
# target_file = 'file.tgt'

# Read in the files so we have an appropriate python dataframe
if tmx_file is not None:
    # tmx files
    metadata, df = tmx2dataframe.read(tmx_file)
else:
    # For 2 parallel files
    df_src = pd.read("file.src", header=None, names=["source_sentence"])
    df_tgt = pd.read("file.tgt", header=None, names=["target_sentence"])
    df = pd.concat([df_src, df_tgt], axis=1)
    df["source_language"] = source_language
    df["target_language"] = target_language

# Have a peak at the data
df.head(3)

ExpatError: not well-formed (invalid token): line 154935, column 46

In [284]:
tmx2dataframe?

Type:        module
String form: <module 'tmx2dataframe.tmx2dataframe' from '/Users/vima/anaconda3/lib/python3.6/site-packages/tmx2dataframe/tmx2dataframe.py'>
File:        ~/anaconda3/lib/python3.6/site-packages/tmx2dataframe/tmx2dataframe.py
Docstring:   <no docstring>


In [ ]:
# This section does the split between train/test/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and 1000 test set. In practice, it's useful to use an external test set


# Do the split between dev/test/train and create parallel corpora
num_dev_patterns = 1000
num_test_patterns = 1000

# Lower case the corpora
df["source_sentence"] = df["source_sentence"].str.lower()
df["target_sentence"] = df["target_sentence"].str.lower()


devtest = df.tail(num_dev_patterns + num_test_patterns)
test = df.tail(num_test_patterns)
dev = df.head(num_dev_patterns)
stripped = df.drop(df.tail(num_dev_patterns + num_test_patterns).index)


stripped[["source_sentence"]].to_csv("train.en", index=False)
stripped[["target_sentence"]].to_csv("train.xh", index=False)

dev[["source_sentence"]].to_csv("dev.en", index=False)
dev[["target_sentence"]].to_csv("dev.xh", index=False)

test[["source_sentence"]].to_csv("test.en", index=False)
test[["target_sentence"]].to_csv("test.xh", index=False)




---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
! export data_path=joeynmt/data/$src$tgt/
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/enxh/vocab.txt

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [ ]:


# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)
name = '%s%s' % (source_language, target_language)

config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "models/{name}_transformer/12000.ckpt" # if given, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"            # Try switching from plateau to Noam scheduling
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100 # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 4000 # Decrease this for testing
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
""".format(name=name, source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [ ]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [ ]:
! cat joeynmt/models/$src$tgt_transformer/validations.txt

In [ ]:
! cd joeynmt; python3 -m joeynmt test models/$src$tgt_transformer/config.yaml
